In [1]:
# imports
import os
import csv
import sys
import tqdm
import joblib
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator

# root = os.path.abspath(os.path.dirname(__file__))
# sys.path.append(root)
pathogen = "abaumannii_organism"
PATH_TO_MODELS = "../other/models"
PATH_TO_OUTPUT = os.path.join(PATH_TO_MODELS, f"model_{pathogen}")
root = os.path.join(PATH_TO_OUTPUT, "framework", "code")

# current file directory
checkpoints_dir = os.path.join(root, "..", "..", "checkpoints")
tasks = pd.read_csv(os.path.join(root, "..", "columns", "run_columns.csv"))['name'].tolist()

# parse arguments
# input_file = sys.argv[1]
# output_file = sys.argv[2]

input_file = os.path.join(root, "..", "examples", "run_input.csv")
output_file = os.path.join(root, "..", "examples", "run_output.csv")

# Read smiles
with open(input_file, "r") as f:
    smiles = []
    reader = csv.reader(f)
    next(reader)
    for r in reader:
        smiles += [r[0]]

# Get Morgan fingerprints
X = []
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=3, fpSize=2048)
for smi in smiles:
    mol = Chem.MolFromSmiles(smi)
    mfp = mfpgen.GetCountFingerprint(mol)
    X.append(mfp.ToList())

# Convert to numpy array
X = np.array(X, dtype=np.int16)

# Create output DataFrame
OUTPUT = pd.DataFrame({"smiles": smiles})  # We will remove this column later

In [2]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)